In [1]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_diagonal_wavelet_full_agriVision.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [ ]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [ ]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,445167.63,-0.63,0.01,-1.17,0.08,0.05,0.05,-0.03,-0.00
3,-0.63,68644.78,3.12,-0.11,0.20,-0.01,0.01,0.00,0.00
4,0.01,3.12,12501.89,0.03,0.10,-0.01,-0.00,-0.00,0.00
5,-1.17,-0.11,0.03,1706.03,0.02,0.02,-0.01,-0.00,-0.00
6,0.08,0.20,0.10,0.02,352.10,0.00,-0.00,0.00,-0.00
7,0.05,-0.01,-0.01,0.02,0.00,92.91,0.00,-0.00,-0.00
8,0.05,0.01,-0.00,-0.01,-0.00,0.00,28.60,-0.00,0.00
9,-0.03,0.00,-0.00,-0.00,0.00,-0.00,-0.00,3.39,-0.00
10,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.18


In [ ]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00000,0.00000,-0.00004,0.00001,0.00001,0.00001,-0.00002,-0.00001
3,-0.00000,1.00000,0.00011,-0.00001,0.00004,-0.00000,0.00001,0.00000,0.00001
4,0.00000,0.00011,1.00000,0.00001,0.00005,-0.00001,-0.00000,-0.00001,0.00001
5,-0.00004,-0.00001,0.00001,1.00000,0.00003,0.00005,-0.00003,-0.00003,-0.00001
6,0.00001,0.00004,0.00005,0.00003,1.00000,0.00002,-0.00003,0.00000,-0.00002
7,0.00001,-0.00000,-0.00001,0.00005,0.00002,1.00000,0.00002,-0.00001,-0.00002
8,0.00001,0.00001,-0.00000,-0.00003,-0.00003,0.00002,1.00000,-0.00003,0.00000
9,-0.00002,0.00000,-0.00001,-0.00003,0.00000,-0.00001,-0.00003,1.00000,-0.00004
10,-0.00001,0.00001,0.00001,-0.00001,-0.00002,-0.00002,0.00000,-0.00004,1.00000


In [ ]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

4.811390603530844

In [ ]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.20280325e+10 5.15551294e+08 1.67436460e+07 3.03594789e+05
 1.24385214e+04 8.14982475e+02 6.75887033e+01 7.19287006e-01
 1.52243484e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,9.997990e-01,-0.019733,-0.003501,-0.000480,-0.000098,-0.000026,-0.000008,-0.000001,-6.013181e-08
1,1.963002e-02,0.999439,-0.026900,-0.003544,-0.000728,-0.000193,-0.000059,-0.000007,-3.545524e-07
2,4.015725e-03,0.026733,0.999352,-0.023256,-0.004692,-0.001240,-0.000381,-0.000045,-2.356896e-06
3,6.362169e-04,0.004112,0.022913,0.998737,-0.043068,-0.010816,-0.003326,-0.000396,-2.116905e-05
4,1.554622e-04,0.001006,0.005533,0.042001,0.996387,-0.070766,-0.020072,-0.002361,-1.263729e-04
5,5.053166e-05,0.000328,0.001802,0.013222,0.067522,0.991141,-0.112979,-0.011749,-6.268075e-04
6,2.147776e-05,0.000140,0.000769,0.005637,0.027424,0.110351,0.991578,-0.061659,-3.220476e-03
7,3.854886e-06,0.000025,0.000136,0.000994,0.004809,0.018235,0.059628,0.996600,-5.363167e-02
8,4.178484e-07,0.000003,0.000014,0.000106,0.000514,0.001948,0.006327,0.053320,9.985554e-01


In [ ]:

cos_dist = spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine')
min_cos_dist = [np.min(row) for row in cos_dist]
arg_min_cos_dist = [np.argmin(row) for row in cos_dist]
min_cos_dist

NameError: name 'group_num' is not defined

In [ ]:
arg_min_cos_dist